<a href="https://colab.research.google.com/github/SaeedehHeydarian/PromptEngineering/blob/main/PromptEngineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install --upgrade langchain

In [93]:
!pip install -qU langchain-groq

In [94]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate
import pandas as pd
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage , HumanMessage
from google.colab import userdata
import os

In [95]:
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

In [96]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    reasoning_format="parsed",
    max_retries=2,
)

## بررسی اثر جزئیات پرامپت بر کیفیت محتوای تولید شده


In [97]:
note="Monthly subscription."

In [98]:
merchant="StarBucks"

In [99]:
template_1="""You are an AI assistant designed to assign a category to a transaction based on the user's transaction note:"{note}" """

In [100]:
prompt_1 = PromptTemplate(template=template_1, input_variables=["note"])

In [101]:
prompt_1=prompt_1.format(note=note)

In [102]:
template_2 = """You are an AI assistant designed to assign a category to a purchase transaction based on the user's transaction note and the merchant name. The user's note is "{note}" and the merchant name is "{merchant}"."""

In [103]:
prompt_2 = PromptTemplate(template=template_2, input_variables=["note" , "merchant"])

In [104]:
prompt_2=prompt_2.format(note=note , merchant=merchant)

In [105]:
template_3 = """You are an AI assistant designed to assign one category from the list below:
-Food and Drink

-Groceries

-House and Garden

-Car and Transportation

-Digital Services

-Health and Fitness

-Entertainment

-Shopping and Retail

-Utilities and Bills

-Travel and Accommodation

-Education and Learning

-Financial Services

-Gifts and Donations

-Personal Care

-Pets

-Insurance

-Rent and Housing

-Kids and Family

-Hobbies and Leisure

Assign the most appropriate category to a purchase transaction based on the user's transaction note and the merchant name. The user's note is "{note}" and the merchant name is "{merchant}"."""


In [106]:
prompt_3 = PromptTemplate(template=template_3, input_variables=["note" , "merchant"])

In [107]:
prompt_3=prompt_3.format(note=note , merchant=merchant)

In [108]:
responses = llm.batch([
    prompt_1,
    prompt_2,
    prompt_3
])

In [109]:
for i in range(len(responses)):
  print(responses[i].content)

**Category:** Subscriptions
Subscriptions
Food and Drink


## تاثیر اثر prompting shot-few در تقلید سبک نوشتاری

In [110]:
prompt_1 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant designed to explain different diseases simply in a scientific style",
        ),
        ("human", "{disease}"),
    ]
)

chain = prompt_1 | llm
result_1=chain.invoke(
    {

        "disease": "Mania",
    }
)
print(result_1.content)

**Mania**  
*A mood‑state disorder that is part of bipolar disorder.*

---

### 1. What it is  
- **Mania** is a period of abnormally elevated, expansive, or irritable mood that lasts at least one week (or less if hospitalization is required).  
- It is one half of the classic bipolar spectrum (the other half being depression).  
- In a manic episode, a person’s energy, activity level, and emotional intensity are far above what is typical for them.

### 2. Core symptoms (DSM‑5 criteria)  
A person must show **at least three** of the following (or four if mood is only irritable):

| Symptom | Typical description |
|---------|---------------------|
| **Elevated or expansive mood** | Feeling “on top of the world,” euphoric, or unusually cheerful. |
| **Increased goal‑directed activity** | Starting many projects, racing thoughts, or feeling “hyper‑productive.” |
| **Reduced need for sleep** | Feeling rested after only a few hours or not needing sleep at all. |
| **Talkativeness / pressured

In [111]:
prompt_second = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an AI assistant designed to explain different diseases simply in a scientific style, this explanation should be short, concise and understnading.
            Explain is and provide the following information:
            1. A brief description of the disease.
            2. key changes (in bullet points).
            3. main types if there is
            2. Symptoms (in bullet points).
            3. Signs (in bullet points).
            4. Who is at risk / what people typically get this disease.
            5. What people should do when they experience it.""",
        ),
        ("human", "{disease}"),
    ]
)
chain = prompt_second | llm
result_2=chain.invoke(
    {

        "disease": "Mania",
    }
).content
print(result_2)

**Mania**  
A mood‑disorder state marked by an abnormally elevated, expansive, or irritable mood, accompanied by increased energy and activity. It is a core feature of bipolar disorder but can also occur in other psychiatric or medical conditions.

**Key physiological/biological changes**  
- ↑ Dopamine and norepinephrine activity in the mesolimbic and prefrontal circuits  
- ↓ GABAergic inhibition → heightened arousal  
- Altered circadian rhythm (shortened sleep, increased wakefulness)  
- ↑ cAMP signaling → changes in neuronal excitability  

**Main types**  
- **Cyclothymic mania** – brief, less intense episodes  
- **Manic episode (Bipolar I)** – ≥1 week of full‑blown mania  
- **Hypomanic episode (Bipolar II)** – ≥4 days, less severe, no psychosis  

**Symptoms** (subjective experiences)  
- Grandiosity or inflated self‑esteem  
- Racing thoughts, rapid speech  
- Decreased need for sleep  
- Increased goal‑directed activity or distractibility  
- Impulsive or risky behavior (spe

## .۳ تاثیر prompting shot-few در تولید خروجی ساختار یافته

In [112]:
!pip install rarfile

In [113]:
import rarfile
with rarfile.RarFile('/content/HW_02_prompt_engineering.rar') as rf:
    rf.extractall('/content/')

In [114]:
with open('/content/resume_sample1.txt', 'r') as f:
    resume_sample1 = f.read()

In [115]:
with open('/content/resume_sample2.txt', 'r') as f:
    resume_sample2 = f.read()

In [116]:
prompt_zero_shot="""
You are an expert resume parser.

Extract information from the following resume and provide it as a JSON object with the exact structure below.
Even if some fields are missing, they must appear in the JSON as empty strings or empty lists.

JSON Structure:

{{
  "full_name": "",
  "contact": {{
    "email": "",
    "phone": ""
  }},
  "education": [
    {{
      "degree": "",
      "field_of_study": "",
      "university": "",
      "graduation_year": ""
    }}
  ],
  "work_experience": [
    {{
      "job_title": "",
      "company": "",
      "duration": "",
      "responsibilities": ""
    }}
  ],
  "skills": [],
  "certifications": []
}}

Resume Text:
{resume_text}

Provide **only the JSON output** in the exact format above. Do not include any explanations or extra text.

"""

In [117]:
prompt_zero_shot=PromptTemplate(template=prompt_zero_shot, input_variables=["resume_text"])

In [118]:
chain_zero=prompt_zero_shot|llm

In [148]:
response_zero_shot=chain_zero.invoke({"resume_text":resume_sample1})

In [149]:
print(response_zero_shot.content)

{
  "full_name": "Sara Moradi",
  "contact": {
    "email": "sara.moradi@example.com",
    "phone": "+98 912 123 4567"
  },
  "education": [
    {
      "degree": "MSc",
      "field_of_study": "Data Science",
      "university": "University of Tehran",
      "graduation_year": "2021"
    },
    {
      "degree": "BSc",
      "field_of_study": "Computer Engineering",
      "university": "Sharif University",
      "graduation_year": "2018"
    }
  ],
  "work_experience": [
    {
      "job_title": "Data Scientist",
      "company": "Snapp",
      "duration": "2022-Present",
      "responsibilities": "Developed machine learning models for customer segmentation Optimized recommendation systems for better user experience"
    },
    {
      "job_title": "AI Researcher",
      "company": "Digikala",
      "duration": "2019-2022",
      "responsibilities": "Conducted NLP research for automatic product categorization"
    }
  ],
  "skills": [
    "Python",
    "TensorFlow",
    "PyTorch",
   

In [150]:
examples = [
    {
        "resume": (
            "Saeedeh Heydarian\n"
            "Email: heydariansaeede0@gmail.com\n"
            "Phone: 09120171150\n"
            "Education: Master Science in Health Information Technology, "
            "Tehran University of Medical Science, 2020\n"
            "Experience: Data Scientist at Asre Danesh, 1 year 10 months\n"
            "Skills: SQL, Python, Machine Learning, Deep Learning\n"
            "Certifications: python maktabkhoone"
        ),
        "result": """{{
  "full_name": "Saeedeh Heydarian",
  "contact": {{"email": "heydariansaeede0@gmail.com", "phone": "09120171150"}},
  "education": [{{
    "degree": "Master Science",
    "field_of_study": "Health Information Technology",
    "university": "Tehran University of Medical Science",
    "graduation_year": "2020"
  }}],
  "work_experience": [{{
    "job_title": "Data Scientist",
    "company": "Asre Danesh",
    "duration": "1 year 10 months",
    "responsibilities": ""
  }}],
  "skills": ["SQL", "Python", "Machine Learning", "Deep Learning"],
  "certifications": ["python maktabkhoone"]
}}"""
    },
    {
        "resume": (
            "Ali Rezaei\n"
            "Email: ali.rezaei@example.com\n"
            "Phone: 09121234567\n"
            "Education: Bachelor in Computer Engineering, "
            "Sharif University of Technology, 2018\n"
            "Experience: Software Engineer at Fanap, 2 years\n"
            "Skills: Python, SQL, Django, REST API\n"
            "Certifications: AWS Certified Solutions Architect"
        ),
        "result": """{{
  "full_name": "Ali Rezaei",
  "contact": {{"email": "ali.rezaei@example.com", "phone": "09121234567"}},
  "education": [{{
    "degree": "Bachelor",
    "field_of_study": "Computer Engineering",
    "university": "Sharif University of Technology",
    "graduation_year": "2018"
  }}],
  "work_experience": [{{
    "job_title": "Software Engineer",
    "company": "Fanap",
    "duration": "2 years",
    "responsibilities": "Developed backend services and APIs using Python and Django"
  }}],
  "skills": ["Python", "SQL", "Django", "REST API"],
  "certifications": ["AWS Certified Solutions Architect"]
}}"""
    }
]

In [151]:
example_prompt = PromptTemplate(
    input_variables=["resume", "result"],
    template="Resume:\n{resume}\n\nResult Output:\n{result}\n"
)

In [152]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=(
        "You are an expert in resume parsing. "
        "Convert the following resume text into structured JSON"
    ),
    suffix="\nNow parse this resume:\n{resume_text}\n:",
    input_variables=["resume_text"]
)

In [153]:
chain_few_shot =few_shot_prompt|llm

In [154]:
Response_Few_shot=response = chain_few_shot.invoke({"resume_text": resume_sample1})

In [155]:
print(Response_Few_shot.content)

```json
{
  "full_name": "Sara Moradi",
  "contact": {
    "email": "sara.moradi@example.com",
    "phone": "+98 912 123 4567"
  },
  "education": [
    {
      "degree": "MSc",
      "field_of_study": "Data Science",
      "university": "University of Tehran",
      "graduation_year": "2021"
    },
    {
      "degree": "BSc",
      "field_of_study": "Computer Engineering",
      "university": "Sharif University",
      "graduation_year": "2018"
    }
  ],
  "work_experience": [
    {
      "job_title": "Data Scientist",
      "company": "Snapp",
      "duration": "2022-Present",
      "responsibilities": "Developed machine learning models for customer segmentation; Optimized recommendation systems for better user experience"
    },
    {
      "job_title": "AI Researcher",
      "company": "Digikala",
      "duration": "2019-2022",
      "responsibilities": "Conducted NLP research for automatic product categorization"
    }
  ],
  "skills": [
    "Python",
    "TensorFlow",
    "PyTo